# Задача 1.

![alt text](zad1.jpg "Zadacha 1")

In [4]:
import numpy as np
import math

In [37]:
# a)
image = np.array([[4,5,2,2,1], [3,3,2,2,4], [4,3,4,1,1], [5,1,4,1,2], [5,1,3,1,4]])
conv_kernel = np.array([[4,3,3], [5,5,5], [2,4,3]])
pading = 0
channel = 1
stride = 1 # stupka

In [38]:
H_output = math.floor(1 + ((image.shape[0] + 2*pading - conv_kernel.shape[0])/stride))

In [39]:
W_output = math.floor(1 + ((image.shape[1] + 2*pading - conv_kernel.shape[1])/stride))

In [40]:
print(f"Razmerite na izhoda shte sa {H_output}x{W_output}")

Razmerite na izhoda shte sa 3x3


In [28]:
np.sum(np.array([[4,5,2], [3,3,2], [4,3,4]]) * conv_kernel)

109

In [41]:
np.sum(np.array([[5,2,2], [3,2,2], [3,4,1]]) * conv_kernel)

92

In [42]:
np.sum(np.array([[2,2,1], [2,2,4], [4,1,1]]) * conv_kernel)

72

In [43]:
np.sum(np.array([[3,3,2], [4,3,4], [5,1,4]]) * conv_kernel)

108

In [44]:
np.sum(np.array([[3,2,2], [3,4,1], [1,4,1]]) * conv_kernel)

85

In [45]:
np.sum(np.array([[2,2,4], [4,1,1], [4,1,2]]) * conv_kernel)

74

In [46]:
np.sum(np.array([[4,3,4], [5,1,4], [5,1,3]]) * conv_kernel)

110

In [47]:
np.sum(np.array([[3,4,1], [1,4,1], [1,3,1]]) * conv_kernel)

74

In [48]:
np.sum(np.array([[4,1,1], [4,1,2], [3,1,4]]) * conv_kernel)

79

In [49]:
gradient = np.ones((3,3))

In [50]:
gradient

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

### Всеки елемент е равен на сбора от елементите на филтъра, с които е участвал в образуването на резултата
#### Централният елемент х3,3: dy/dx33 = сумата от целия филтър, тъй като взима участие в образуването на всеки елемент от резултата, с всеки елемент от филтъра

In [51]:
derivate_image = np.array([[4, 7, 10, 6, 3], [9, 17, 25, 16, 8], [11, 23, 34, 23, 11], [7, 16, 24, 17, 8], [2, 6, 9, 7, 3]])

In [52]:
derivate_image

array([[ 4,  7, 10,  6,  3],
       [ 9, 17, 25, 16,  8],
       [11, 23, 34, 23, 11],
       [ 7, 16, 24, 17,  8],
       [ 2,  6,  9,  7,  3]])

# Задача 2

![alt text](zad2.jpg "Zadacha 2")

# Задача 3.

In [1]:
import os

In [2]:
trainig_set_dir = r'/home/nasko/Desktop/NBU_MAG/sem2/Machine_Learning_II/kontrolno2/datasets'

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends as cudnn
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import pickle

In [4]:
data_transforms = {
    'train' : transforms.Compose([transforms.RandomResizedCrop(224),
                                  transforms.RandomHorizontalFlip(),
                                  transforms.ToTensor(),
                                  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
}

data_dir = trainig_set_dir
image_datasets = {'train': datasets.ImageFolder(os.path.join(data_dir, 'train'),
                    data_transforms['train'])}
dataloaders = {'train' : torch.utils.data.DataLoader(image_datasets['train'], batch_size = 4, shuffle = True, num_workers = 4)}

In [5]:
dataset_size = {'train' : len(image_datasets['train'])}
class_names = image_datasets['train'].classes

In [6]:
!ls '/home/nasko/Desktop/NBU_MAG/sem2/Machine_Learning_II/kontrolno2/datasets'

train  val


In [7]:
num_categories = 33

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
device

device(type='cpu')

In [10]:
model_ft = models.vgg16(pretrained=True)

In [11]:
model_ft

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [12]:
model_ft.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [13]:
num_features = model_ft.classifier[0].out_features

In [14]:
# otstranqvane na poslednite dva sloq
model_ft.classifier = nn.Sequential(*list(model_ft.classifier.children())[:3])

In [15]:
model_ft.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
)

In [16]:
model_ft.classifier.append(nn.Linear(num_features, num_categories))

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=33, bias=True)
)

In [17]:
# Spirane update na veche treniranite layeri
for param in model_ft.features.parameters():
    param.require_grad = False

In [18]:
# Trenirane na posledniq layer
training_layer = list(model_ft.classifier.children())[-1]

model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()

# Nastroika na optimizatora
optimizer_ft = optim.SGD(training_layer.parameters(), lr = 0.001, momentum = 0.9)

# Namalqvane na learning rate, za da ne se podmine globalen minimum
lr_scheduled = lr_scheduler.StepLR(optimizer_ft, step_size = 7, gamma = 0.01)

In [19]:
def train_model(model, criterion, optimizer, scheduler, dataloaders, num_epochs = 25):

    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(1, num_epochs+1):
        print(f"Epoch: {epoch}/{num_epochs}")
        print('-'*10)

        phase = 'train'

        model.train()

        running_loss = 0.0
        running_corrects = 0
        i = 0
        for inputs, labels in dataloaders['train']:
            if i % 100 == 0:
                print(f"{i}th Iteration")
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            i+=1
        scheduler.step()
        epoch_loss = running_loss / dataset_size['train']
        epoch_acc = running_corrects.double() / dataset_size['train']

        print(f"{phase} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f} |")

        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

        print()
        
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [21]:
model_ft = train_model(model_ft, criterion, optimizer_ft, lr_scheduled, dataloaders, num_epochs = 5)

Epoch: 1/5
----------
0th Iteration
100th Iteration
200th Iteration
300th Iteration
400th Iteration
500th Iteration
600th Iteration
700th Iteration
800th Iteration
900th Iteration
1000th Iteration


KeyboardInterrupt: 

In [118]:
len(dataloaders['train'])

4214

In [124]:
0 % 100

0

In [142]:
torch.cuda.is_available()

False